In [1]:
import pandas as pd
import numpy as np
import os
import gradio as gr
from dotenv import load_dotenv

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain.document_loaders import DataFrameLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain import hub
from langchain.schema import AIMessage, HumanMessage, SystemMessage

from langchain_core.runnables import (
    ConfigurableField,
    RunnableBinding,
    RunnableLambda,
    RunnablePassthrough,
    RunnableParallel
)

pd.set_option('display.max_colwidth', 300)  # Or use a large number if 'None' does not work in some environments
pd.set_option('display.max_columns', 300)  # Show all columns
pd.set_option('display.max_rows', 20)  # Show all row

load_dotenv()
OPENAI_APIKEY = os.environ['OPENAI_APIKEY']

# Try out tools
-https://python.langchain.com/docs/modules/tools/

In [2]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [3]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=100)
tool = WikipediaQueryRun(api_wrapper=api_wrapper)

In [4]:
print(tool.name)
print(tool.description)
print(tool.input_schema)
print(tool.args)
print(tool.return_direct)
print(tool.output_schema)

wikipedia
A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.
<class 'pydantic.v1.main.wikipediaSchema'>
{'query': {'title': 'Query', 'type': 'string'}}
False
<class 'pydantic.v1.main.wikipedia_output'>


In [5]:
tool.invoke("langchain")

'Page: LangChain\nSummary: LangChain is a framework designed to simplify the creation of applications '

In [6]:
tool.run("langchain")

'Page: LangChain\nSummary: LangChain is a framework designed to simplify the creation of applications '

In [7]:
from langchain_core.pydantic_v1 import BaseModel, Field


class WikiInputs(BaseModel):
    """Inputs to the wikipedia tool."""

    query: str = Field(
        description="query to look up in Wikipedia, should be 3 or less words"
    )

In [8]:
tool = WikipediaQueryRun(
    name="wiki-tool",
    description="look up things in wikipedia",
    args_schema=WikiInputs,
    api_wrapper=api_wrapper,
    return_direct=True,
)

In [9]:
from langchain_community.tools.pubmed.tool import PubmedQueryRun

tool = PubmedQueryRun()


In [10]:
tool.args


{'query': {'title': 'Query', 'type': 'string'}}

# Build Custom Tools

In [11]:
# Import things that are needed generically
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool

In [12]:
@tool
def search(query: str) -> str:
    """Look up things online."""
    return "LangChain"

In [13]:
print(search.name)
print(search.description)
print(search.args)

search
search(query: str) -> str - Look up things online.
{'query': {'title': 'Query', 'type': 'string'}}


In [14]:
search.invoke("title")

'LangChain'

In [15]:
@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return f"The answer is: {a * b}"

In [16]:
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
multiply(a: int, b: int) -> int - Multiply two numbers.
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


In [17]:
multiply.invoke({'a':1, 'b':2})

'The answer is: 2'

In [18]:
class SearchInput(BaseModel):
    query: str = Field(description="should be a search query")


@tool("search-tool", args_schema=SearchInput, return_direct=True)
def search(query: str) -> str:
    """Look up things online."""
    return "LangChain"

In [19]:
print(search.args)

{'query': {'title': 'Query', 'description': 'should be a search query', 'type': 'string'}}


# Tools from Structured Data Tools


In [20]:
def search_function(query: str):
    return "LangChain"


search = StructuredTool.from_function(
    func=search_function,
    name="Search",
    description="useful for when you need to answer questions about current events",
    # coroutine= ... <- you can specify an async method if desired as well
)

In [21]:
print(search.name)
print(search.description)
print(search.args)

Search
Search(query: str) - useful for when you need to answer questions about current events
{'query': {'title': 'Query', 'type': 'string'}}


# Tool with error handling

In [22]:
from langchain_core.tools import ToolException

class CalculatorInput(BaseModel):
    a: int = Field(description="first number")
    b: int = Field(description="second number")
    c: int = Field(description="third number")


def multiply(a: int, b: int, c: int) -> int:
    """Multiply two numbers."""
    try:
        return a * b + 'c'  #put a mistake here
    except:
        raise ToolException("This search tool is not available")


calculator = StructuredTool.from_function(
    func=multiply,
    name="Calculator",
    description="multiply numbers",
    args_schema=CalculatorInput,
    return_direct=True,
    handle_tool_error=True
    # coroutine= ... <- you can specify an async method if desired as well
)

In [23]:
calculator.invoke({'a': 1, 'b': 2, 'c': 3})

'This search tool is not available'

# Tools as OpenAI Functions

In [24]:
from langchain_core.messages import HumanMessage
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain_openai import ChatOpenAI
from langchain_community.tools import MoveFileTool


In [25]:
calculator

StructuredTool(name='Calculator', description='Calculator(a: int, b: int, c: int) -> int - multiply numbers', args_schema=<class '__main__.CalculatorInput'>, return_direct=True, handle_tool_error=True, func=<function multiply at 0x12de7a520>)

In [26]:
model = ChatOpenAI(api_key=OPENAI_APIKEY, model="gpt-3.5-turbo")



In [27]:
tools = [MoveFileTool()]
functions = [convert_to_openai_function(t) for t in tools]
functions[0]


{'name': 'move_file',
 'description': 'Move or rename a file from one location to another',
 'parameters': {'type': 'object',
  'properties': {'source_path': {'description': 'Path of the file to move',
    'type': 'string'},
   'destination_path': {'description': 'New path for the moved file',
    'type': 'string'}},
  'required': ['source_path', 'destination_path']}}

In [28]:
message = model.invoke(
    [HumanMessage(content="move file foo to bar")], functions=functions
)

In [29]:
message

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"source_path":"foo","destination_path":"bar"}', 'name': 'move_file'}}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 77, 'total_tokens': 97}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-0549b5d0-d480-4fa8-82ba-649d839d2827-0')

In [30]:
message.additional_kwargs

{'function_call': {'arguments': '{"source_path":"foo","destination_path":"bar"}',
  'name': 'move_file'}}

In [31]:
# Don't need to put the human message class here and it still works
message = model.invoke(
    "move file from foo to bar", functions=functions
)

In [32]:
message.additional_kwargs

{'function_call': {'arguments': '{"source_path":"foo","destination_path":"bar"}',
  'name': 'move_file'}}

In [33]:
message

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"source_path":"foo","destination_path":"bar"}', 'name': 'move_file'}}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 78, 'total_tokens': 98}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-d8570eee-9f96-4dad-b801-cf69aadf8a8f-0')

In [34]:
# Use openai to automatically bind functions
model_with_functions = model.bind_functions(tools)
model_with_functions.invoke("move file foo to bar")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"source_path":"foo","destination_path":"bar"}', 'name': 'move_file'}}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 77, 'total_tokens': 97}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-b7c9fe46-0922-4d48-b9aa-52b1826d7eae-0')

In [35]:
model_with_functions.invoke("Tell me a joke")

AIMessage(content="Why couldn't the bicycle find its way home? Because it lost its bearings!", response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 76, 'total_tokens': 93}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-8b7ea077-e66c-44d5-a1d2-b23de24a1ea5-0')

Not sure the difference between model with functions vs model with tools and binding the tool

In [36]:
# Use OpenAI tools and tool choice here too
model_with_tools = model.bind_tools(tools)
model_with_tools.invoke([HumanMessage(content="move file foo to bar")])

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_7yWO3AH4pJ1FXrVdWrri6DKe', 'function': {'arguments': '{"source_path":"foo","destination_path":"bar"}', 'name': 'move_file'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 77, 'total_tokens': 97}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-8bec4525-2665-4268-962c-767cb09e4842-0', tool_calls=[{'name': 'move_file', 'args': {'source_path': 'foo', 'destination_path': 'bar'}, 'id': 'call_7yWO3AH4pJ1FXrVdWrri6DKe'}])

In [37]:
# Use OpenAI tools and tool choice here too
# See it knows when to use the tool!!
model_with_tools = model.bind_tools(tools)
model_with_tools.invoke([HumanMessage(content="tell me a joke")])

AIMessage(content="Why don't scientists trust atoms?\n\nBecause they make up everything!", response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 76, 'total_tokens': 90}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-8e874787-f4a5-4027-889e-72b9092669fe-0')

In [38]:
model_with_tools = model.bind_tools(tools)
model_with_tools.invoke([HumanMessage(content="Move the file from foo")])

AIMessage(content='I will need the source path and the destination path to move the file. Could you please provide me with this information?', response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 77, 'total_tokens': 102}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-f60b543a-75b6-4f15-9336-9923595fd693-0')

Wow it even knew it couldn't call the tool becuase i didnt give the destination path!

# Call multiple tool

In [76]:
# Define tools
class WaterQuestionInput(BaseModel):
    gender: str = Field(description="Answer with Male, Female, or Other")
    age: int


@tool("waterquestion-tool", args_schema=WaterQuestionInput, return_direct=True)
def WaterQuestion(gender: str, age: int ) -> str:
    """Look up water recommendations based on age and gender"""
    answer = f"For {gender} aged {age}, you should drink 8 cups a day"
    return answer

class ProteinQuestionInput(BaseModel):
    gender: str = Field(description="Answer with Male, Female, or Other")
    age: int
    weight: float

@tool("proteinquestion-tool", args_schema=ProteinQuestionInput, return_direct=True)
def ProteinQuestion(gender: str, age: int, weight: int ) -> str:
    """Look up water recommendations based on age, gender, and weight."""
    answer = f"For {gender} aged {age} weighing {weight}, you should \
         eat 1g of protein per kg."
    return answer


In [77]:
# Convert to openai functions
water_function = convert_to_openai_function(WaterQuestion)
protein_function = convert_to_openai_function(ProteinQuestion)
tools = [water_function, protein_function]


In [78]:
# Instantiate Model
model = ChatOpenAI(api_key=OPENAI_APIKEY, model="gpt-4-turbo")


In [79]:
# Bind the tools
model_with_tools = model.bind_tools(tools)

In [80]:
# Try it out
model_with_tools.invoke("How much water should I drink?")

AIMessage(content='To provide you with accurate information on how much water you should drink, I need to know your age and gender. Could you please provide that information?', response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 153, 'total_tokens': 184}, 'model_name': 'gpt-4-turbo', 'system_fingerprint': 'fp_294de9593d', 'finish_reason': 'stop', 'logprobs': None}, id='run-ae1dcbad-9009-4a64-bb55-6000d7adbed5-0')

note!! Wtih gpt 3.5 it hallucinates the inputs - making up age and gender.

In [81]:
model_with_tools.invoke("How much water should I drink as a female aged 30?")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_p9rvWO9m6HH942fgsyQxNhE8', 'function': {'arguments': '{"gender":"Female","age":30}', 'name': 'waterquestion-tool'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 159, 'total_tokens': 178}, 'model_name': 'gpt-4-turbo', 'system_fingerprint': 'fp_294de9593d', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-d368b5a6-af1e-46c2-8c54-1e4397381e33-0', tool_calls=[{'name': 'waterquestion-tool', 'args': {'gender': 'Female', 'age': 30}, 'id': 'call_p9rvWO9m6HH942fgsyQxNhE8'}])

In [88]:
model_with_tools.invoke("How much protein should I eat as a male, aged 30, weighing 200 lbs?")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_lDeU1547IimJEWD7AsEgOmI7', 'function': {'arguments': '{"gender":"Male","age":30,"weight":200}', 'name': 'proteinquestion-tool'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 165, 'total_tokens': 189}, 'model_name': 'gpt-4-turbo', 'system_fingerprint': 'fp_ea6eb70039', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-d99297fe-6d8b-43a4-ae13-f591e3a83e98-0', tool_calls=[{'name': 'proteinquestion-tool', 'args': {'gender': 'Male', 'age': 30, 'weight': 200}, 'id': 'call_lDeU1547IimJEWD7AsEgOmI7'}])

# Agent Executor
- Use an agent to actually execute on the prompts

In [82]:
from langchain.agents import AgentExecutor, create_openai_functions_agent


In [83]:
# The prompt must have an agent scratch pad
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant"),
        MessagesPlaceholder("chat_history", optional=True),
        ("human", "{input}"),
        MessagesPlaceholder("agent_scratchpad"),
    ]
)


In [87]:
# Don't convert the tools to the openai format before putting them into the
# openai agent
tools = [ProteinQuestion, WaterQuestion]

In [88]:
# First construct an agent
# Construct the OpenAI Functions agent
agent = create_openai_functions_agent(model, tools, prompt)

In [89]:
# Construct something to execute on the agent
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [90]:
agent_executor.invoke({'input': 'How much protein should a female aged 30 eat?'})



> Entering new AgentExecutor chain...
To provide an accurate protein recommendation, I need to know the weight of the individual. Could you please provide the weight of the 30-year-old female?

> Finished chain.


{'input': 'How much protein should a female aged 30 eat?',
 'output': 'To provide an accurate protein recommendation, I need to know the weight of the individual. Could you please provide the weight of the 30-year-old female?'}

In [91]:
agent_executor.invoke({'input': 'How much protein should a female aged 30 weighing 120 lbs eat?'})



> Entering new AgentExecutor chain...

Invoking: `proteinquestion-tool` with `{'gender': 'Female', 'age': 30, 'weight': 120}`


For Female aged 30 weighing 120.0, you should          eat 1g of protein per kg.


> Finished chain.


{'input': 'How much protein should a female aged 30 weighing 120 lbs eat?',
 'output': 'For Female aged 30 weighing 120.0, you should          eat 1g of protein per kg.'}